# Cluster
### Objectives
- Fit and evaluate a cluster model to group similar data
- Understand the profile for each cluster
### Inputs
- outputs/datasets/collection/house-price-2021.csv
- Instructions on which variables to use for data cleaning and feature engineering. They are found in their respective notebooks.
### Outputs
- Cluster Pipeline
- Train Set
- Most important features to define a cluster plot
- Clusters Profile Description
- Cluster Silhouette
---
## Change working directory

We need to change the working directory from its current folder to its parent folder

- We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the parent of current directory the new current directory

    - os.path.dirname() gets the parent directory
    - os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## **Load Data**

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/house-price-2021.csv")
      .drop(['WoodDeckSF', 'SalePrice', 'EnclosedPorch'], axis=1)
      )
print(df.shape)
df.head(3)

## Cluster Pipeline with all data
### ML Cluster Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.compose import ColumnTransformer
from feature_engine.encoding import OrdinalEncoder
from feature_engine.imputation import MeanMedianImputer
from feature_engine.transformation import LogTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Define numerical features
numerical_features = ['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr',
                    'BsmtFinSF1', 'BsmtUnfSF','LotArea',
                    'GarageArea',  'GarageYrBlt', 'OpenPorchSF',
                    'LotFrontage', 'MasVnrArea',  'OverallCond',	
                    'OverallQual', 'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd']  # List of numerical features

# Define categorical features
categorical_features = ['KitchenQual','BsmtExposure','BsmtFinType1','GarageFinish']  # List of categorical features

def PipelineCluster():
    # Define ordinal encoder for categorical features
    ordinal_encoder = OrdinalEncoder(encoding_method='arbitrary', variables=categorical_features)

    # Define correlated feature selection
    correlated_selection = SmartCorrelatedSelection(method="spearman", threshold=0.6, selection_method="variance")

    # Define imputer for missing values
    imputer = SimpleImputer(strategy='median')

    # Define scaler for numerical features
    scaler = StandardScaler()

    # Define PCA for dimensionality reduction
    pca = PCA(n_components=50, random_state=101)

    # Define KMeans clustering
    kmeans = KMeans(n_clusters=50, random_state=101)

    # Define pipeline steps
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", ordinal_encoder),
        ("SmartCorrelatedSelection", correlated_selection),
        ("imputer", imputer),
        ("scaler", scaler),
        ("PCA", pca),
        ("model", kmeans)
    ])
    
    return pipeline_base

### Principal Component Analysis (PCA)

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_pca = Pipeline(pipeline_cluster.steps[:-2])
df_pca = pipeline_pca.fit_transform(df)

print(df_pca.shape,'\n', type(df_pca))


Apply PCA separately to the scaled data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

n_components = 15


def pca_components_analysis(df_pca, n_components):
    pca = PCA(n_components=n_components).fit(df_pca)
    x_PCA = pca.transform(df_pca)  # array with transformed PCA

    ComponentsList = ["Component " + str(number)
                      for number in range(n_components)]
    dfExplVarRatio = pd.DataFrame(
        data=np.round(100 * pca.explained_variance_ratio_, 3),
        index=ComponentsList,
        columns=['Explained Variance Ratio (%)'])

    dfExplVarRatio['Accumulated Variance'] = dfExplVarRatio['Explained Variance Ratio (%)'].cumsum(
    )

    PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum(
    )

    print(
        f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n")
    plt.figure(figsize=(8, 6))
    sns.lineplot(data=dfExplVarRatio,  marker="D")
    plt.xticks(rotation=90)
    plt.yticks(np.arange(0, 110, 10))
    plt.show()


pca_components_analysis(df_pca=df_pca, n_components=n_components)

In [ ]:
pca_components_analysis(df_pca=df_pca, n_components=8)

In [ ]:
def PipelineCluster():
    # Define ordinal encoder for categorical features
    ordinal_encoder = OrdinalEncoder(encoding_method='arbitrary', variables=categorical_features)

    # Define correlated feature selection
    correlated_selection = SmartCorrelatedSelection(method="spearman", threshold=0.6, selection_method="variance")

    # Define imputer for missing values
    imputer = SimpleImputer(strategy='median')

    # Define scaler for numerical features
    scaler = StandardScaler()

    # Define PCA for dimensionality reduction
    pca = PCA(n_components=9, random_state=101)

    # Define KMeans clustering
    kmeans = KMeans(n_clusters=50, random_state=101)

    # Define pipeline steps
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", ordinal_encoder),
        ("SmartCorrelatedSelection", correlated_selection),
        ("imputer", imputer),
        ("scaler", scaler),
        ("PCA", pca),
        ("model", kmeans)
    ])
    
    return pipeline_base

### Elbow Method and Silhouette Score

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_analysis = Pipeline(pipeline_cluster.steps[:-1])
df_analysis = pipeline_analysis.fit_transform(df)

print(df_analysis.shape,'\n', type(df_analysis))

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt

# Available font families
plt.rcParams['font.family'] = 'DejaVu Sans'

visualizer = KElbowVisualizer(KMeans(random_state=101), k=(1,11)) # 11 is not included, it will plot until 10
visualizer.fit(df_analysis) 
visualizer.show() 
plt.show()

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

# 9 is not included, it will stop at 8
n_cluster_start, n_cluster_stop = 3, 9

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(KMeans(random_state=101), k=(
    n_cluster_start, n_cluster_stop), metric='silhouette')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(estimator=KMeans(n_clusters=n_clusters, random_state=101),
                                      colors='yellowbrick')
    visualizer.fit(df_analysis)
    visualizer.show()
    plt.show()
    print("\n")

In [ ]:
def PipelineCluster():
    # Define ordinal encoder for categorical features
    ordinal_encoder = OrdinalEncoder(encoding_method='arbitrary', variables=categorical_features)

    # Define correlated feature selection
    correlated_selection = SmartCorrelatedSelection(method="spearman", threshold=0.6, selection_method="variance")

    # Define imputer for missing values
    imputer = SimpleImputer(strategy='median')

    # Define scaler for numerical features
    scaler = StandardScaler()

    # Define PCA for dimensionality reduction
    pca = PCA(n_components=8, random_state=101)

    # Define KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=101)

    # Define pipeline steps
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", ordinal_encoder),
        ("SmartCorrelatedSelection", correlated_selection),
        ("scaler", scaler),
        ("imputer", imputer),
        ("PCA", pca),
        ("model", kmeans)
    ])
    
    return pipeline_base

### Fit Cluster Pipeline
Quick recap of our data for training cluster pipeline

In [ ]:
X = df.copy()
print(X.shape)
X.head()

Fit Cluster pipeline

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

### Add cluster predictions to dataset
We add a column *Clusters* (with the cluster pipeline predictions) to the dataset

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_
print(X.shape)
X.head(5)

In [ ]:
print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_analysis[:, 0], y=df_analysis[:, 1],
                hue=X['Clusters'], palette='bright', alpha=0.6)
plt.scatter(x=pipeline_cluster['model'].cluster_centers_[:, 0], y=pipeline_cluster['model'].cluster_centers_[:, 1],
            marker="*", s=140, linewidths=1, color="black")
plt.xlabel("PCA Component 0")
plt.ylabel("PCA Component 1")
plt.title("PCA Components colored by Clusters")
plt.show()

We save the cluster predictions from this pipeline to use in the future. We will get back to that in a later stage.

In [ ]:
cluster_predictions_with_all_variables = X['Clusters']
cluster_predictions_with_all_variables

### Fit a classifier, where the target is cluster predictions and features remaining variables
We copy X to a DataFrame df_clf

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df_clf.drop(['Clusters'], axis=1),
    df_clf['Clusters'],
    test_size=0.2,
    random_state=0
)

print(X_train.shape, X_test.shape)

Create classifier pipeline steps

In [ ]:
# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithm
from sklearn.ensemble import GradientBoostingClassifier

imputer = SimpleImputer(strategy='median')

def PipelineClf2ExplainClusters():
    # Define ordinal encoder for categorical features
    ordinal_encoder = OrdinalEncoder(encoding_method='arbitrary', variables=categorical_features)

    # Define correlated feature selection
    correlated_selection = SmartCorrelatedSelection(method="spearman", threshold=0.6, selection_method="variance")

    # Define scaler for numerical features
    scaler = StandardScaler()

    # Define feature selection using GradientBoostingClassifier
    feature_selection = SelectFromModel(GradientBoostingClassifier(random_state=101))

    # Define GradientBoostingClassifier
    classifier = GradientBoostingClassifier(random_state=101)

    # Define pipeline steps
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", ordinal_encoder),
        ("SmartCorrelatedSelection", correlated_selection),
        ("scaler", scaler),
        ("imputer", imputer),
        ("feat_selection", feature_selection),
        ("model", classifier)
    ])
    
    return pipeline_base


PipelineClf2ExplainClusters()

Fit the classifier to the training data

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster.fit(X_train, y_train)

Sanity check to observe if there are any missing values in the data.

In [ ]:
print(X_train.isnull().sum(), y_train.isnull().sum())

Evaluate classifier performance on the Train and Test Sets

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

The *classification_report()* function in scikit-learn generates a comprehensive report of classification metrics, such as precision, recall, F1-score, and support, for each class in the target variable.

In [ ]:
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

### Assess the most important Features in the dataset that define a cluster

In [ ]:
# after data cleaning and feature engineering, the feature space changes
# This pipeline has two data cleaning and feature engineering steps 

data_cleaning_feat_eng_steps = 2
columns_after_data_cleaning_feat_eng = (Pipeline(pipeline_clf_cluster.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support(
)].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support()],
    'Importance': pipeline_clf_cluster['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# reassign best features in importance order
best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{best_features} \n")
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

The best_features are listed below and are stored in the variable *best_features_pipeline_all_variables*

In [ ]:
best_features_pipeline_all_variables = best_features
best_features_pipeline_all_variables

### Cluster Analysis
Load function that plots a table with description for all Clusters

In [ ]:
def DescriptionAllClusters(df, decimal_points=3):

    DescriptionAllClusters = pd.DataFrame(
        columns=df.drop(['Clusters'], axis=1).columns)
    # iterate on each cluster , calls Clusters_IndividualDescription()
    for cluster in df.sort_values(by='Clusters')['Clusters'].unique():

        EDA_ClusterSubset = df.query(
            f"Clusters == {cluster}").drop(['Clusters'], axis=1)
        ClusterDescription = Clusters_IndividualDescription(
            EDA_ClusterSubset, cluster, decimal_points)
        DescriptionAllClusters = pd.concat([DescriptionAllClusters, ClusterDescription])

    DescriptionAllClusters.set_index(['Cluster'], inplace=True)
    return DescriptionAllClusters


def Clusters_IndividualDescription(EDA_Cluster, cluster, decimal_points):

    ClustersDescription = pd.DataFrame(columns=EDA_Cluster.columns)
    # for a given cluster, iterate over all columns
    # if the variable is numerical, calculate the IQR: display as Q1 -- Q3.
    # That will show the range for the most common values for the numerical variable
    # if the variable is categorical, count the frequencies and displays the top 3 most frequent
    # That will show the most common levels for the category

    for col in EDA_Cluster.columns:

        try:  # eventually a given cluster will have only missing data for a given variable

            if EDA_Cluster[col].dtypes == 'object':

                top_frequencies = EDA_Cluster.dropna(
                    subset=[col])[[col]].value_counts(normalize=True).nlargest(n=3)
                Description = ''

                for x in range(len(top_frequencies)):
                    freq = top_frequencies.iloc[x]
                    category = top_frequencies.index[x][0]
                    CategoryPercentage = int(round(freq*100, 0))
                    statement = f"'{category}': {CategoryPercentage}% , "
                    Description = Description + statement

                ClustersDescription.at[0, col] = Description[:-2]

            elif EDA_Cluster[col].dtypes in ['float', 'int']:
                DescStats = EDA_Cluster.dropna(subset=[col])[[col]].describe()
                Q1 = round(DescStats.iloc[4, 0], decimal_points)
                Q3 = round(DescStats.iloc[6, 0], decimal_points)
                Description = f"{Q1} -- {Q3}"
                ClustersDescription.at[0, col] = Description

        except Exception as e:
            ClustersDescription.at[0, col] = 'Not available'
            print(
                f"** Error Exception: {e} - cluster {cluster}, variable {col}")

    ClustersDescription['Cluster'] = str(cluster)

    return ClustersDescription

Load custom function to plot cluster distribution per Variable 

In [ ]:
import plotly.express as px


def cluster_distribution_per_variable(df, target):
    """
    The data should have 2 variables, the cluster predictions and
    the variable you want to analyze with, in this case we call "target".
    We use plotly express to create 2 plots:
    Cluster distribution across the target.
    Relative presence of the target level in each cluster.
    """
    df_bar_plot = df.value_counts(["Clusters", target]).reset_index()
    df_bar_plot.columns = ['Clusters', target, 'Count']
    df_bar_plot[target] = df_bar_plot[target].astype('object')

    print(f"Clusters distribution across {target} levels")
    fig = px.histogram(df_bar_plot, x='Clusters', y='Count',
                 color=target, width=800, height=500)
    fig.update_layout(xaxis=dict(tickmode='array',
                      tickvals=df['Clusters'].unique()))
    fig.show(renderer='colab')

    df_relative = (df
                   .groupby(["Clusters", target])
                   .size()
                   .unstack()
                   .groupby(level=0)
                   .apply(lambda x:  100*x / x.sum())
                   .stack()
                   .reset_index(name='Relative Percentage (%)')
                   .sort_values(by=['Clusters'])
                   )
    df_relative.columns = ['Clusters', target, 'Relative Percentage (%)']

    print(f"Relative Percentage (%) of {target} in each cluster")
    fig = px.line(df_relative, x='Clusters', y='Relative Percentage (%)',
                  color=target, width=800, height=500)
    fig.update_layout(xaxis=dict(tickmode='array',
                      tickvals=df['Clusters'].unique()))
    fig.update_traces(mode='markers+lines')
    fig.show(renderer='colab')

Create a DataFrame that contains best features and Clusters Predictions since we want to analyse the patterns for each cluster.

In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
print(df_cluster_profile.shape)
df_cluster_profile.head(3)

We want also to analyse SalePrices.

In [ ]:
df_SP = pd.read_csv("outputs/datasets/collection/house-price-2021.csv").filter(['SalePrice'])
df_SP['SalePrice'] = df_SP['SalePrice'] 
df_SP.head(3)

#### Cluster profile based on the best features

This represents the range of values for various features within each cluster, including the years built, basement finished square footage, unfinished square footage, total basement square footage, garage finish type distribution (RFn, Unf, Fin), and sale prices, with corresponding percentages for each category.

**Cluster 0:** represents houses between 1976 - 2005, with basement finished square footage ranging from 0.0 to 528.0, unfinished square footage ranging from 474.0 to 1252.0, total basement square footage ranging from 840.0 to 1405.0, a garage finish distribution of 'RFn': 54%, 'Unf': 28%, and 'Fin': 18%. Finally Sale prices fall between 160000.0 and 214000.0.

**Cluster 1:** Has houses between 1996 - 2006, with basement finished square footage ranging from 759.0 to 1218.0, unfinished square footage ranging from 116.0 to 438.0, total basement square footage ranging from 1054.0 to 1639.0, a garage finish distribution of 'Fin': 68%, 'Unf': 24%, and 'RFn': 8%. Finally Sale prices fall between 195575.0 and 301875.0.

**Cluster 2:** showcases properties between 1922 - 1965, with basement finished square footage ranging from 0.0 to 539.0, unfinished square footage ranging from 168.0 to 684.0, total basement square footage ranging from 684.0 to 1003.0, a garage finish distribution of 'Unf': 70%, 'None': 11%, and 'Fin': 10%. Finally Sale prices fall between 110500.0 and 155000.0. 

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile = DescriptionAllClusters(df=pd.concat([df_cluster_profile, df_SP], axis=1), decimal_points=0)
clusters_profile

In [ ]:
# Display the first few rows of df_cluster_profile
print("First few rows of df_cluster_profile:")
print(df_cluster_profile.head())

# Display the shape of df_cluster_profile (number of rows and columns)
print("\nShape of df_cluster_profile:")
print(df_cluster_profile.shape)

# Display information about df_cluster_profile, including data types and missing values
print("\nInformation about df_cluster_profile:")
print(df_cluster_profile.info())

# Display summary statistics for numeric columns in df_cluster_profile
print("\nSummary statistics for df_cluster_profile:")
print(df_cluster_profile.describe())

# Repeat the above steps for df_SP
print("\nFirst few rows of df_SP:")
print(df_SP.head())

print("\nShape of df_SP:")
print(df_SP.shape)

print("\nInformation about df_SP:")
print(df_SP.info())

print("\nSummary statistics for df_SP:")
print(df_SP.describe())


#### Clusters distribution across Churn levels & Relative Percentage of SalePrice in each cluster

In [ ]:
df_cluster_vs_SP = df_SP.copy()
df_cluster_vs_SP['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_SP, target='SalePrice')

## Fit New Cluster Pipeline with most important features
In order to reduce feature space, we will study the trade-off between the previous Cluster Pipeline (fitted with all variables) and Pipeline using the variables that are most important to define the clusters from the previous pipeline

In [ ]:
best_features_pipeline_all_variables

### Define trade-off and metrics to compare new and previous Cluster Pipeline
To evaluate this trade-off we will

1. Conduct a elbow method and silhouette analysis and check if the same number of clusters is suggested
2. Fit new cluster pipeline and compare if the predictions from this pipeline are "equivalent" to the predictions from the previous pipeline
3. Fit a classifier to explain cluster, and check if performance on Train and Test sets is similar to the previous pipeline
4. Check if the most important features for the classifier are the same from the previous pipeline
5. ompare if the cluster profile from both pipelines are "equivalent"

If we are happy to say **yes** for them, we can use a cluster pipeline using the features that best define the clusters from previous pipeline!

* The **gain** is that in real-time (which is the major purpose of Machine Learning) you will need fewer variables for predicting clusters for your prospects.

### Subset data with the most relevant variables

In [ ]:
df_reduced = df.filter(best_features_pipeline_all_variables)
df_reduced.head(3)

Rewrite Cluster Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

def PipelineCluster():
    # Define numerical and categorical features
    numerical_features = ['YearBuilt', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF'] 
    categorical_features = ['GarageFinish']  

    # Define transformers for numerical and categorical features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    # Combine numerical and categorical transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', numeric_transformer, numerical_features),
            ('categorical', categorical_transformer, categorical_features)])

    # Define KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=0)

    # Define pipeline steps
    pipeline_base = Pipeline([
        ('preprocessor', preprocessor),
        ('model', kmeans)
    ])
    
    return pipeline_base

# Create the pipeline
pipeline = PipelineCluster()

# Display the pipeline
pipeline


### Apply Elbow Method and Silhouette analysis

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_analysis = Pipeline(pipeline_cluster.steps[:-1])
df_analysis = pipeline_analysis.fit_transform(df_reduced)

print(df_analysis.shape,'\n', type(df_analysis))

Elbow Analysis

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11))
visualizer.fit(df_analysis) 
visualizer.show() 
plt.show()

This code visualizes the silhouette scores and plots for different numbers of clusters to aid in selecting the optimal number of clusters for KMeans clustering.

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

n_cluster_start, n_cluster_stop = 2, 5

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(
    n_cluster_start, n_cluster_stop), metric='silhouette')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(estimator=KMeans(n_clusters=n_clusters, random_state=0),
                                      colors='yellowbrick')
    visualizer.fit(df_analysis)
    visualizer.show()
    plt.show()
    print("\n")

### Fit New Cluster Pipeline
We set X as our training set for the cluster. It is a copy of df_reduced

In [ ]:
X = df_reduced.copy()
print(X.shape)
X.head(3)

Fit Cluster pipeline

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

### Add cluster predictions to dataset
We add a column "*Clusters*" (with the cluster pipeline predictions) to the dataset

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_
print(X.shape)
X.head(3)

In [ ]:
print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

### Compare current cluster predictions to previous cluster predictions
We just fitted a new cluster pipeline and want to compare if its predictions are "*equivalent*" to the previous cluster.

These are the predictions from the **previous** cluster pipeline - trained with all variables

In [ ]:
cluster_predictions_with_all_variables

And these are the predictions from **current** cluster pipeline (trained with ['OnlineBackup', 'MonthlyCharges', 'PhoneService'])

In [ ]:
cluster_predictions_with_best_features = X['Clusters'] 
cluster_predictions_with_best_features

We use a confusion matrix to evaluate if the predictions of both pipelines are "**equivalent**"

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(cluster_predictions_with_all_variables, cluster_predictions_with_best_features))

### Fit a classifier, where the target is cluster predictions and features remaining variables

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_clf.drop(['Clusters'], axis=1),
    df_clf['Clusters'],
    test_size=0.2,
    random_state=101
)

print(X_train.shape, X_test.shape)
df_clf.columns

Rewrite pipeline to explain clusters

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import GradientBoostingClassifier

def PipelineClf2ExplainClusters():
    # Define categorical features
    categorical_features = ['GarageFinish']

    # Define transformers for categorical features
    categorical_transformer = OrdinalEncoder(categories='auto', handle_unknown='use_encoded_value', unknown_value=-1)

    # Define scaler for numerical features
    scaler = StandardScaler()

    # Define GradientBoostingClassifier
    classifier = GradientBoostingClassifier(random_state=101)

    # Define pipeline steps
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", categorical_transformer),
        ("scaler", scaler),
        ("model", classifier)
    ])
    
    return pipeline_base

# Test the pipeline
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster.fit(X_train, y_train)


### Fit a classifier, where the target is cluster labels and features remaining variables

Create and fit a classifier pipeline to learn the feature importance when defining a cluster

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster.fit(X_train, y_train)

Evaluate classifier performance on Train and Test Sets

In [ ]:
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

In [ ]:
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

### Assess Most Important Features

In [ ]:
# since we don't have feature selection step in this pipeline, best_features is X_train columns
best_features = X_train.columns.to_list()

# create a DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': best_features,
    'Importance': pipeline_clf_cluster['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

### Cluster Analysis
Create a DataFrame that contains the best features and Clusters Predictions: we want to analyse the patterns for each cluster.

In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
df_cluster_profile.head(3)

We want also to analyse SalePrice levels

In [ ]:
df_SP = pd.read_csv("outputs/datasets/collection/house-price-2021.csv").filter(['SalePrice'])
df_SP['SalePrice'] = df_SP['SalePrice']
df_SP.head(3)

### Cluster profile on most important features

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile = DescriptionAllClusters(df= pd.concat([df_cluster_profile, df_SP], axis=1), decimal_points=0)
clusters_profile

---

### Clusters distribution across SalePrice levels & Relative Percentage of SalePrice in each cluster

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Generate the plot
df_cluster_vs_SP = df_SP.copy()
df_cluster_vs_SP['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_SP, target='SalePrice')

# Save the plot to a file
plt.savefig('cluster_distribution.png')

# Display the saved image
Image(filename='cluster_distribution.png')
image_path = 'cluster_distribution.png'
display(Image(filename=image_path))


In [ ]:
pipeline_cluster

## Push files to Repo
We will generate the following files

* Cluster Pipeline
* Train Set
* Feature importance plot
* Clusters Description
* Cluster Silhouette

In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/cluster_analysis/{version}'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

### Cluster pipeline

In [ ]:
pipeline_cluster

In [ ]:
joblib.dump(value=pipeline_cluster, filename=f"{file_path}/cluster_pipeline.pkl")

## Train Set

In [ ]:
print(df_reduced.shape)
df_reduced.head(3)

In [ ]:
df_reduced.to_csv(f"{file_path}/TrainSet.csv", index=False)

## Most important features plot
These are the features that define a cluster

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance', figsize=(8,4))
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance', figsize=(8,4))
plt.savefig(f"{file_path}/features_define_cluster.png", bbox_inches='tight', dpi=150)

## Cluster Profile

In [ ]:
clusters_profile

In [ ]:
clusters_profile.to_csv(f"{file_path}/clusters_profile.csv")

## Cluster silhouette plot

In [ ]:
visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0], colors='yellowbrick')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(7,5))
fig = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0], colors='yellowbrick', ax=axes)
fig.fit(df_analysis)

plt.savefig(f"{file_path}/clusters_silhouette.png", bbox_inches='tight', dpi=150)